# Advent Of Code 2023

## Day 1: Trebuchet?!

In [ ]:
# Part 1
def extract_number(from_line: str) -> int:
    clean_line = ''.join(char for char in from_line if char.isdigit())
    return int(clean_line[0] + clean_line[-1])

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

In [ ]:
# Part 2
DIGIT_REPLACEMENT = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

def extract_number(from_line: str) -> int:
    
    first_digit, last_digit = None, None

    for (index, char) in enumerate(from_line):
        
        # Match regular digit
        if char.isdigit():
            last_digit = char
        
        else:
            # Match spelled out digit
            matches = [value for digit, value in DIGIT_REPLACEMENT.items() if from_line[index:].startswith(digit)]
            if any(matches):
                last_digit = str(matches[0])
    
        if first_digit is None:
            first_digit = last_digit
            
    return int(first_digit + last_digit)

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

## Day 2 - Cube conundrum

In [ ]:
# Part 1
import re

MAX_AMOUNTS = {
    "red":   12,
    "green": 13,
    "blue":  14,
}

RE_PARSE_GAME = re.compile(r"Game (\d+): (.*)")
RE_PARSE_DRAW = re.compile(r"(\d+) (red|green|blue)")

def process_game(line: str) -> tuple[int, int]:
    """Process the game result given as a string
    Returns a tuple containing the id of the game and True if the game is possible.
    """
    game_id, game_content = RE_PARSE_GAME.match(line).groups()

    return int(game_id), all(
        int(number) <= MAX_AMOUNTS[color]
        for number, color in (
            RE_PARSE_DRAW.search(bit).groups()
            for draw in game_content.split(sep=";")
            for bit in draw.split(sep=",")
        )
    )

with open('files/day2-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all possible game IDs is {sum(id for id, result in (process_game(line) for line in input_file) if result)}")

In [ ]:
# Part 2
import re
import operator
from functools import reduce

RE_PARSE_GAME = re.compile(r"Game (\d+): (.*)")
RE_PARSE_DRAW = re.compile(r"(\d+) (red|green|blue)")

def process_game_power(line: str) -> int:
    """Process the game result given as a string
    Returns the power of the minimal possible set of cubes for the game.
    """
    game_id, game_content = RE_PARSE_GAME.match(line).groups()

    min_amounts = dict()
    for number, color in (RE_PARSE_DRAW.search(bit).groups() for draw in game_content.split(sep=";") for bit in draw.split(sep=",")):
        min_amounts[color] = max(int(number), min_amounts.get(color, 0))
        
    return reduce(operator.mul, min_amounts.values(), 1)

with open('files/day2-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all possible game IDs is {sum(process_game_power(line) for line in input_file)}")

## Day 3 - Gear ratios

In [ ]:
# Part 1
import re

RE_FIND_NUMBER = re.compile(r"\d+")

with open('files/day3-input', encoding='utf-8') as input_file:
    engine: tuple[str, ...] = tuple(line.strip() for line in input_file)

# Compute boundaries
ENGINE_HEIGHT = len(engine)
ENGINE_WIDTH = len(engine[0])

total = 0

for row, line in enumerate(engine):
    for match in RE_FIND_NUMBER.finditer(line):
        if any(
            char != "." and not char.isdigit()
            for subline in engine[max(0, row - 1) : min(ENGINE_HEIGHT, row + 2)]
            for char in subline[max(0, match.start() - 1) : min(ENGINE_WIDTH, match.end() + 1)]
        ):
            total += int(match.group(0))

print(f"The sum of all part number is {total}")

In [ ]:
# Part 2
import re

RE_FIND_NUMBER = re.compile(r"\d+")

with open('files/day3-input', encoding='utf-8') as input_file:
    engine: tuple[str, ...] = tuple(line.strip() for line in input_file)

# Compute boundaries
ENGINE_HEIGHT = len(engine)
ENGINE_WIDTH = len(engine[0])

# This dictionary uses as key the position of a gear, and holds as value the list of part number adjacent to that gear
gears: dict[tuple[int, int], list[int]] = dict()

# Check every number for adjacent gear ; add it to gears dictionary
for row, line in enumerate(engine):
    for match in RE_FIND_NUMBER.finditer(line):
        for i in range(max(0, row - 1), min(ENGINE_HEIGHT, row + 2)):
            for j in range(max(0, match.start() - 1), min(ENGINE_WIDTH, match.end() + 1)):
                if engine[i][j] == "*":
                    gears[(i, j)] = gears.get((i, j), []) + [int(match.group(0))]


print(f"The sum of all part number is {sum(parts[0] * parts[1] for parts in gears.values() if len(parts) == 2)}")

## Day 4 - Scratchcards

In [ ]:
# Part 1
import re

RE_SCRATCHCARD = re.compile(r"Card\s+(\d+):\s*((?:\d+\s*)*) \| ((?:\s*\d+)*)\s*")

def compute_scratchcard(card: str) -> int:
    """Computes the score of a scratchcard given its string representation"""
    card_id, winning_numbers, your_numbers = RE_SCRATCHCARD.match(card).groups()
    return int(2**(sum(1 for number in your_numbers.split() if number and number in winning_numbers.split()) - 1))
    
with open('files/day4-input', encoding='utf-8') as input_file:
    print(f"The sum of all the scratchcards score is {sum(compute_scratchcard(line) for line in input_file)}")
    

In [ ]:
# Part 2
import re

RE_SCRATCHCARD = re.compile(r"Card\s+(\d+):\s*((?:\d+\s*)*) \| ((?:\s*\d+)*)\s*")

# This dictionary stores the number of each scratchcard (<card ID>: <number of cards>)
count_scratchcards: dict[int, int] = dict()

def compute_scratchcard(card: str) -> None:
    """Computes the scratchcards won by a scratchcard given its string representation"""
    card_id, winning_numbers, your_numbers = RE_SCRATCHCARD.match(card).groups()
    card_id = int(card_id)
    
    # Compute score
    score = sum(1 for number in your_numbers.split() if number and number in winning_numbers.split())
    
    # Update counts
    count_scratchcards[card_id] = count_scratchcards.get(card_id, 0) + 1
    for i in range(card_id + 1, card_id + score + 1):
        count_scratchcards[i] = count_scratchcards.get(i, 0) + count_scratchcards[card_id]
    
with open('files/day4-input', encoding='utf-8') as input_file:
    [compute_scratchcard(line) for line in input_file]
    print(f"The sum of all the scratchcards score is {sum(count_scratchcards.values())}")
    

## Day 5 - If You Give A Seed A Fertilizer

In [ ]:
# Part 1
import re

RE_READ_SEEDS = re.compile(r"\d+")
RE_READ_MAP = re.compile(r"(?P<destination>\d+) (?P<source>\d+) (?P<length>\d+)", flags=re.MULTILINE)

class TransformMap:
    
    def __init__(self, string_map):
        self.ranges = [
            (range(source, source + length), destination - source)
            for destination, source, length in map(lambda match: map(int, match), RE_READ_MAP.findall(string_map))
        ]
        
    def __getitem__(self, value):
        for source, offset in self.ranges:
            if value in source:
                return value + offset
        else:
            return value


with open('files/day5-input', encoding='utf-8') as input_file:
    seeds, *maps = input_file.read().split(sep="\n\n")

seeds = [int(seed) for seed in RE_READ_SEEDS.findall(seeds)]
maps = [TransformMap(map_str) for map_str in maps]

results = seeds[:]
for tmap in maps:
    results = [tmap[i] for i in results]

print(f"The lowest location number where a seed can be planted is {min(results)}")

In [ ]:
# Part 2
import re

RE_READ_SEEDS = re.compile(r"\d+")
RE_READ_MAP = re.compile(r"(?P<destination>\d+) (?P<source>\d+) (?P<length>\d+)", flags=re.MULTILINE)

class TransformMap:
    
    def __init__(self, string_map):
        self.ranges = [
            (range(source, source + length), destination - source)
            for destination, source, length in map(lambda match: map(int, match), RE_READ_MAP.findall(string_map))
        ]
        
    def applyToRange(self, r_from):
        
        ranges_from = [r_from]
        ranges_to = []
        
        for range_source, offset in self.ranges:
            for range_from in ranges_from[:]:
                intersection = range(max(range_from.start, range_source.start), min(range_from.stop, range_source.stop))
                if not intersection.start < intersection.stop:
                    continue
                    
                ranges_to.append(range(intersection.start + offset, intersection.stop + offset))
                
                # Replace the range_from by anything not in the intersection (on the left or on the right) in ranges_from
                ranges_from.remove(range_from)
                
                if range_from.start < intersection.start:
                    ranges_from.append(range(range_from.start, intersection.start))
                    
                if intersection.stop < range_from.stop:
                    ranges_from.append(range(intersection.stop, range_from.stop))
        
        # Return all ranges_to + what is left untouched in ranges_from
        return ranges_to + ranges_from

with open('files/day5-input', encoding='utf-8') as input_file:
    seeds, *maps = input_file.read().split(sep="\n\n")

seeds = [int(seed) for seed in RE_READ_SEEDS.findall(seeds)]
seeds = [range(start, start + length) for start, length in zip(seeds[::2], seeds[1::2])]

t_maps = [TransformMap(map_str) for map_str in maps]

results = seeds
for t_map in t_maps:
    results = [r for x in results for r in t_map.applyToRange(x)]
    
print(f"The lowest location number where a seed can be planted is {min(r.start for r in results)}")

## Day 6 - Wait For It

In [ ]:
# Part 1
import re
from functools import reduce
import operator as op
from typing import Callable

RE_NUMBER = re.compile(r"\d+")

def modelize_race(duration: int) -> Callable[[int], [int]]:
    """This function returns another function which modelize a race
    for a given duration. The returned function takes the time spent
    pressing the button and returns the distance reached
    """
    return lambda charge: charge * (duration - charge)

with open('files/day6-input', encoding='utf-8') as input_file:
    races = [
        (int(time), int(record))
        for time, record in zip(RE_NUMBER.findall(input_file.readline()), RE_NUMBER.findall(input_file.readline()))
    ]

nb_winning_ways = [
    sum(1 for duration in range(time) if modelize_race(time)(duration) > record)
    for time, record in races
]

print(f"Multiplying the number of ways to win each race yields {reduce(op.mul, nb_winning_ways, 1)}")

In [ ]:
# Part 2

with open('files/day6-input', encoding='utf-8') as input_file:
    time = int(RE_NUMBER.search(input_file.readline().replace(" ", "")).group())
    record = int(RE_NUMBER.search(input_file.readline().replace(" ", "")).group())
    
race = modelize_race(time)

print(f"The record is beatable in {sum(1 for duration in range(time) if race(duration) > record)} ways")

## Day 7 - Camel Cards

In [ ]:
# Part 1
from collections import Counter
from enum import Enum

FACE_CARDS = {
    'A': 14,
    'K': 13,
    'Q': 12,
    'J': 11,
    'T': 10,
}

class CamelHand:
    """This class represents a hand of Camel Cards.
    The index of the hand is a base-14 integer with 6 (base-14) digits.
    The most significant digit represent the hand type, while every
    other digit in order of significance represent the cards.
    This index makes sorting the hands by value trivial.
    """
    
    class HandTypes(Enum):
        FIVE_OF_A_KIND  = 6
        FOUR_OF_A_KIND  = 5
        FULL_HOUSE      = 4
        THREE_OF_A_KIND = 3
        TWO_PAIRS       = 2
        ONE_PAIR        = 1
        HIGH_CARD       = 0
    
    def __init__(self, hand):
        self.hand = [FACE_CARDS[card] if card in FACE_CARDS else int(card) for card in hand]
        self.index = sum(x * 14**n for n, x in enumerate(self.hand[::-1])) + self.get_hand_type().value * 14**5
    
    def get_hand_type(self):
        _, counts = zip(*Counter(self.hand).most_common())
        
        match counts[0]:
            case 5:
                return CamelHand.HandTypes.FIVE_OF_A_KIND
            case 4:
                return CamelHand.HandTypes.FOUR_OF_A_KIND
            case 3:
                return CamelHand.HandTypes.FULL_HOUSE if counts[1] == 2 else CamelHand.HandTypes.THREE_OF_A_KIND
            case 2:
                return CamelHand.HandTypes.TWO_PAIRS if counts[1] == 2 else CamelHand.HandTypes.ONE_PAIR
            case _:
                return CamelHand.HandTypes.HIGH_CARD
        

with open('files/day7-input', encoding='utf-8') as input_file:
    games = [(CamelHand(hand), int(bet)) for (hand, bet) in (line.split() for line in input_file)]
    
print(f"The total winnings are {sum(bet * (i + 1) for i, (hand, bet) in enumerate(sorted(games, key=lambda game: game[0].index)))}")

In [ ]:
# Part 2
from collections import Counter
from enum import Enum

FACE_CARDS = {
    'A': 14,
    'K': 13,
    'Q': 12,
    'T': 10,
    'J': 1,
}

class CamelHand:
    """This class represents a hand of Camel Cards.
    The index of a hand is a base-14 integer with 6 (base-14) digits.
    The most significant digit represent the hand type, while every
    other digit in descending order of significance represent the cards.
    This index makes sorting the hands by value trivial.
    """
    
    class HandTypes(Enum):
        FIVE_OF_A_KIND  = 6
        FOUR_OF_A_KIND  = 5
        FULL_HOUSE      = 4
        THREE_OF_A_KIND = 3
        TWO_PAIRS       = 2
        ONE_PAIR        = 1
        HIGH_CARD       = 0
    
    def __init__(self, hand):
        self.hand = [FACE_CARDS[card] if card in FACE_CARDS else int(card) for card in hand]
        self.index = sum(x * 14**n for n, x in enumerate((*self.hand[::-1], self.get_hand_type().value)))
        # if 'J' in hand:
        #     print(f"{hand} is {self.get_hand_type()}")
    
    def get_hand_type(self):
        counter = Counter(self.hand) # We don't count jokers here because they do not form hands by themselves
                
        if counter[FACE_CARDS['J']] == 5:
            # Edge case: only jokers in hand is a Five-of-a-king
            return CamelHand.HandTypes.FIVE_OF_A_KIND
        
        _, counts = zip(*((card, count) for (card, count) in counter.most_common() if card != FACE_CARDS['J']))
        
        match counts[0] + counter[FACE_CARDS['J']]:
            case 5:
                return CamelHand.HandTypes.FIVE_OF_A_KIND
            case 4:
                return CamelHand.HandTypes.FOUR_OF_A_KIND
            case 3:
                return CamelHand.HandTypes.FULL_HOUSE if len(counts) > 1 and counts[1] == 2 else CamelHand.HandTypes.THREE_OF_A_KIND
            case 2:
                return CamelHand.HandTypes.TWO_PAIRS if len(counts) > 1 and counts[1] == 2 else CamelHand.HandTypes.ONE_PAIR
            case _:
                return CamelHand.HandTypes.HIGH_CARD
        

with open('files/day7-input', encoding='utf-8') as input_file:
    games = [(CamelHand(hand), int(bet)) for (hand, bet) in (line.split() for line in input_file)]
    
print(f"The total winnings are {sum(bet * (i + 1) for i, (hand, bet) in enumerate(sorted(games, key=lambda game: game[0].index)))}")

## Day 8 - Haunted Wasteland

In [ ]:
# Part 1
import re
from itertools import cycle

RE_NODE = re.compile(r"(?P<Node>\w{3}) = \((?P<L>\w{3}), (?P<R>\w{3})\)")

START_NODE = "AAA"
END_NODE = "ZZZ"

with open('files/day8-input', encoding='utf-8') as input_file:
    _1, _2 = input_file.read().split(sep="\n\n")
    instructions = cycle(_1)
    map_network = {node['Node']: node for node in (RE_NODE.match(line).groupdict() for line in _2.split(sep="\n"))}

current_node = START_NODE
nb_steps = 0

while current_node != END_NODE:
    current_node = map_network[current_node][next(instructions)]
    nb_steps += 1

print(f"{nb_steps} step{'s' if nb_steps > 1 else ''} are required to reach {END_NODE}")

In [ ]:
# Part 2
import re
from itertools import cycle
from functools import reduce
import math

RE_NODE = re.compile(r"(?P<Node>\w{3}) = \((?P<L>\w{3}), (?P<R>\w{3})\)")

with open('files/day8-input', encoding='utf-8') as input_file:
    instructions, map_str = input_file.read().split(sep="\n\n")
    map_network = {node['Node']: node for node in (RE_NODE.match(line).groupdict() for line in map_str.split(sep="\n"))}

start_nodes = [node for node in map_network if node[-1] == 'A']
end_nodes = [node for node in map_network if node[-1] == 'Z']

def run(start_node):
    current_node = start_node
    nb_steps = 0
    
    for move in cycle(instructions):
        if current_node in end_nodes:
            return nb_steps
        
        current_node = map_network[current_node][move]
        nb_steps += 1


print(f"It takes {reduce(math.lcm, (run(node) for node in start_nodes), 1)} steps before we're only on nodes that end with a Z")